# Notebook pour la question de programmation du Devoir 2


### Objectives d'apprentissage
Dans ce problème, nous allons implémenter la régression logistique et la tester sur un jeu de données d'analyse des sentiments.

### Code à écrire
Recherchez le mot clé "TODO" et rajoutez votre code dans l'espace vide indiqué entre les commentaires du début et fin de votre solution.

### Pré-traitement des données

#### Classe et fonction pour charger les données

In [121]:
# Importer des bibliothèques
import argparse
import time

# Définir une classe pour stocker un seul exemple (instance) de sentiment (words, label)
class SentimentExample:
    def __init__(self, words, label):
        self.words = words
        self.label = label

    def __repr__(self):
        return repr(self.words) + "; label=" + repr(self.label)

    def __str__(self):
        return self.__repr__()


# Lit les exemples de sentiments au format [0 ou 1]<TAB>[phrase brute] ; tokenise et nettoie les phrases.
def read_sentiment_examples(infile):
    f = open(infile, encoding='iso8859')
    exs = []
    for line in f:
            fields = line.strip().split(" ")
            label = 0 if "0" in fields[0] else 1
            exs.append(SentimentExample(fields[1:], label))
    f.close()
    return exs

#### *Charger* les données

In [122]:
# Monter le lecteur pour accéder aux fichiers dans gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [123]:
#VOUS DEVEZ CHANGER LE CHEMIN DE train_file ET dev_file SELON OÙ VOUS LES STOCKEZ DANS VOTRE gdrive.

#"TODO" changez le chemin pour train_file
train_file = '/content/gdrive/MyDrive/IFT714/tp2/train.txt'
#"TODO" changez le chemin pour dev_file
dev_file = '/content/gdrive/MyDrive/IFT714/tp2/dev.txt'

# Charger les données des fichiers
train_exs = read_sentiment_examples(train_file)
dev_exs = read_sentiment_examples(dev_file)
n_pos = 0
n_neg = 0
for ex in train_exs:
    if ex.label == 1:
        n_pos += 1
    else:
        n_neg += 1
print("%d train examples: %d positive, %d negative" % (len(train_exs), n_pos, n_neg))
print("%d dev examples" % len(dev_exs))


6920 train examples: 3610 positive, 3310 negative
872 dev examples


#### Indexer les exemples
Cette section contient le code d'un indexeur (Indexer) qui est utile pour créer un mappage entre les mots et les index. Il a déjà été implémenté pour vous. Lisez-le et essayez de comprendre ce que font ses méthodes.

In [124]:
# Bijection entre objets et entiers commençant à 0. Utile pour le mappage
# étiquettes, attributs(features), etc. en coordonnées d'un espace vectoriel.

# Cette classe crée un mapping entre des objets (ici des mots) et des index uniques
# Par exemple : apple->1, banana->2, etc.

class Indexer(object):
    def __init__(self):
        self.objs_to_ints = {}
        self.ints_to_objs = {}

    def __repr__(self):
        return str([str(self.get_object(i)) for i in range(0, len(self))])

    def __str__(self):
        return self.__repr__()

    def __len__(self):
        return len(self.objs_to_ints)

    # Renvoie l'objet correspondant à l'index particulier
    def get_object(self, index):
        if (index not in self.ints_to_objs):
            return None
        else:
            return self.ints_to_objs[index]

    def contains(self, object):
        return self.index_of(object) != -1

    # Renvoie -1 si l'objet n'est pas présent, l'index sinon
    def index_of(self, object):
        if (object not in self.objs_to_ints):
            return -1
        else:
            return self.objs_to_ints[object]

    # Ajoute l'objet à l'index s'il n'est pas présent, renvoie toujours un index non négatif
    def add_and_get_index(self, object, add=True):
        if not add:
            return self.index_of(object)
        if (object not in self.objs_to_ints):
            new_idx = len(self.objs_to_ints)
            self.objs_to_ints[object] = new_idx
            self.ints_to_objs[new_idx] = object
        return self.objs_to_ints[object]

### Definir le modèle de régression logistique

In [125]:
# Importer des bibliothèques
import sys
from collections import Counter
from typing import List
import numpy as np
import math
import random

#### Definir l'extracteur d'attributs/features

In [126]:
# Type de base d'extraction d'attributs (features). Prend un exemple et renvoie une liste indexée d'attributs.
class FeatureExtractor(object):
    # Extraire les attributs (features). Inclut un indicateur add_to_indexer pour contrôler si l'indexeur doit être étendu.
    # Au moment du test, tous les attributs/features inconnus (non-vu auparavant) doivent être ignorés, mais au moment de l'entrainement,
    # nous voulons probablement continuer à l'étendre.
    def extract_features(self, ex, add_to_indexer):
        raise Exception("Don't call me, call my subclasses")


# Extrait les attributs (features) unigramme sac-de-mots (BOW) d'une phrase.
# Vous pouvez considérer un unigramme comme un mot unique (par exemple "love", "you").
# C'est à vous de décider comment vous voulez gérer les comptages
class UnigramFeatureExtractor(FeatureExtractor):
    def __init__(self, indexer: Indexer):
        self.indexer = indexer

    def extract_features(self, ex, add_to_indexer=False):
        #features = Counter()
        features = np.zeros(len(self.indexer))
        for w in ex.words:
            feat_idx = self.indexer.add_and_get_index(w) \
                if add_to_indexer else self.indexer.index_of(w)
            if feat_idx != -1:
                if len(features) > feat_idx:
                  features[feat_idx] += 1.0
                else: 
                  features = np.append(features, 1)
        return features

In [127]:
data = train_exs[0:8]
random.shuffle(data)
print(data)

[['this', 'is', 'a', 'visually', 'stunning', 'rumination', 'on', 'love', ',', 'memory', ',', 'history', 'and', 'the', 'war', 'between', 'art', 'and', 'commerce', '.']; label=1, ['a', 'fan', 'film', 'that', 'for', 'the', 'uninitiated', 'plays', 'better', 'on', 'video', 'with', 'the', 'sound', 'turned', 'down', '.']; label=0, ['they', 'presume', 'their', 'audience', 'wo', "n't", 'sit', 'still', 'for', 'a', 'sociology', 'lesson', ',', 'however', 'entertainingly', 'presented', ',', 'so', 'they', 'trot', 'out', 'the', 'conventional', 'science-fiction', 'elements', 'of', 'bug-eyed', 'monsters', 'and', 'futuristic', 'women', 'in', 'skimpy', 'clothes', '.']; label=0, ['jonathan', 'parker', "'s", 'bartleby', 'should', 'have', 'been', 'the', 'be-all-end-all', 'of', 'the', 'modern-office', 'anomie', 'films', '.']; label=1, ['campanella', 'gets', 'the', 'tone', 'just', 'right', '--', 'funny', 'in', 'the', 'middle', 'of', 'sad', 'in', 'the', 'middle', 'of', 'hopeful', '.']; label=1, ['bÃ©art', 'and

#### Definir les classifieurs de base

In [128]:
# Type de base du classificateur de sentiment
class SentimentClassifier(object):
    # Makes a prediction for the given example
    def predict(self, ex: SentimentExample):
        raise Exception("Don't call me, call my subclasses")


# Prédit toujours la classe positive
class AlwaysPositiveClassifier(SentimentClassifier):
    def predict(self, ex: SentimentExample):
        return 1

#### Classe régression logistique

In [129]:
class LogisticRegressionClassifier(SentimentClassifier):
    def __init__(self, feat_extractor: FeatureExtractor, train_examples, num_iters=150, reg_lambda=0.0, learning_rate=0.1, batch_size=100):
        # TODO : Initialiser le modèle de régression logistique
        
        # Arguments : feat_extractor est unigram, train_examples est un jeu de données d'entrainement
        # num_iters est le nombre d'époques, reg_lambda est le paramètre de régularisation
        # learning_rate est le taux d'apprentissage utilisé dans la descente de gradient
        
        # ÉTAPE 1 : Définissez les variables pour les poids et les biais, et initialisez-les à zéro
        
        # ÉTAPE 2 : Appelez la fonction train(). (Cela a déjà été fait pour vous)

        ##### DÉBUT DE LA SOLUTION #####
        self.w = np.zeros(len(feat_extractor.indexer))
        self.b = 0
        self.feat_extractor = feat_extractor
        self.train_exs = train_examples
        self.num_iters = num_iters
        self.reg_lambda = reg_lambda
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        ##### FIN DE LA SOLUTION  #####

        self.train()


    def train(self):
        # TODO : fonction d'entraînement du modèle de régression logistique.
        # Utilisez une descente de gradient stochastique.
        

        ##### DÉBUT DE LA SOLUTION #####
        sig = np.vectorize(self.sigmoid)
        ex_labels = np.vectorize(self.extract_label)
        loss_func = np.vectorize(self.bce)

        for epoch in range(self.num_iters):
          loss = 0
          shuffled_train_examples = self.train_exs
          random.shuffle(shuffled_train_examples)
          for idx in range(int(len(shuffled_train_examples) / self.batch_size)):
            train_batch = shuffled_train_examples[idx*self.batch_size:(idx+1)*self.batch_size]
            train_data = np.asarray([self.feat_extractor.extract_features(data) for data in train_batch])
            train_label = np.asarray(ex_labels(shuffled_train_examples[idx*self.batch_size:(idx+1)*self.batch_size]))
            # Forward pass
            out = sig(self.w.T@train_data.T+self.b)
            # Calculate Loss
            loss -= (1/self.batch_size)*sum(loss_func(out, train_label)) #add for printing loss or adding to mean loss over epochs
            # Backpropagate Through network
            self.backprop(out, train_label, train_data)
          loss /= int(len(shuffled_train_examples) / self.batch_size)
          b = "epoch : " + str(epoch) + " Current Loss : " + str(loss)
          sys.stdout.write('\r'+b)


        ##### FIN DE LA SOLUTION  #####    
    
    def sigmoid(self, x):
      # Returns the sigmoid of x
      return 1/(1+math.exp(-x))

    def extract_label(self, train_ex: SentimentExample):
      # extracts the label from the training sentence
      return train_ex.label

    def bce(self, pred, label):
      # calculates the binary cross entropy loss between two variables
      return -label*math.log2(pred)+(1-label)*math.log2(1-pred) 

    def backprop(self, out, train_label, train_batch):
      # Backpropagates over network
      self.w = self.w - self.learning_rate*(1/self.batch_size)*(out-train_label)@train_batch
      self.b = self.b - self.learning_rate*np.mean((out-train_label))

    def predict(self, ex):
        # TODO : prédiction du modèle de régression logistique pour un seul exemple
        ##### DÉBUT DE LA SOLUTION #####
        input = np.asarray(self.feat_extractor.extract_features(ex))
        return round(self.sigmoid(self.w.T@input.T+self.b))
        ##### FIN DE LA SOLUTION  #####

#### Fonction d'entrainement pour la régression logisique

In [130]:
# Entrainer un modèle de régression logsitique sur les exemples d'entrainement en utilisant le FeatureExtractor, tous donnés en paramètres.
def train_lr(train_exs: List[SentimentExample], feat_extractor: FeatureExtractor, reg_lambda) -> LogisticRegressionClassifier:
    # TODO : fonction d'entraînement du modèle de régression logistique.
    # Remplissez le feature_extractor.
    # Initialisez et renvoiez un objet d'instance LogisticRegressionClassifier
    
    ##### DÉBUT DE LA SOLUTION #####
    # Remplir feature extractor
    for ex in train_exs:
      feat_extractor.extract_features(ex, add_to_indexer=True)

    learning_rate = 0.1 # add loop with different learning rates to try to optimise hyperparameters???
    return LogisticRegressionClassifier(feat_extractor, train_exs, learning_rate=learning_rate)
    ##### FIN DE LA SOLUTION  #####

In [131]:
# POINT D'ENTREE PRINCIPAL pour vos modifications. 
# Entraîne et retourne un des modèles possibles selon les options passées.
def train_model(feature_type, model_type, train_exs, reg_lambda=0.0):
    
    # Initialize feature extractor
    if feature_type == "unigram":
        feat_extractor = UnigramFeatureExtractor(Indexer())
    else:
        raise Exception("Pass unigram")

    # Train the model
    if model_type == "AlwaysPositive":
        model = AlwaysPositiveClassifier()
    elif model_type == "LogisticRegression":
        model = train_lr(train_exs, feat_extractor, reg_lambda=reg_lambda)
    else:
        raise Exception("Pass AlwaysPositive or LogisticRegression")
    return model

### Fonctions pour l'évaluation du modèle

In [132]:
# Évalue un classificateur donné sur les exemples donnés
def evaluate(classifier, exs):
    return print_evaluation([ex.label for ex in exs], [classifier.predict(ex) for ex in exs])


# Imprime la précision en comparant la vérité du terrain (ground truth) - golds - et les prédictions, chacune étant une séquence d'étiquettes 0/1.
def print_evaluation(golds, predictions):
    num_correct = 0
    num_pos_correct = 0
    num_pred = 0
    num_gold = 0
    num_total = 0
    if len(golds) != len(predictions):
        raise Exception("Mismatched gold/pred lengths: %i / %i" %
                        (len(golds), len(predictions)))
    for idx in range(0, len(golds)):
        gold = golds[idx]
        prediction = predictions[idx]
        if prediction == gold:
            num_correct += 1
        if prediction == 1:
            num_pred += 1
        if gold == 1:
            num_gold += 1
        if prediction == 1 and gold == 1:
            num_pos_correct += 1
        num_total += 1

    print("Accuracy: %i / %i = %.2f %%" %
          (num_correct, num_total,
           num_correct * 100.0 / num_total))
    return num_correct * 100.0 / num_total
    
# ENTRÉE PRINCIPALE POUR L'ÉVALUATION sur les jeux de données d'entrainement et de développement
def eval_train_dev(model):
    print("===== Train Accuracy =====")
    train_acc = evaluate(model, train_exs)
    print("===== Dev Accuracy =====")
    eval_acc = evaluate(model, dev_exs)
    return [train_acc, eval_acc]

### Évaluation du modèle avec une représentation unigramme Bag-of-Words

In [ ]:
# Évaluer la régression logistique avec des attributs unigrammes

lr_unigram_model = train_model('unigram', 'LogisticRegression', train_exs)
eval_train_dev(lr_unigram_model)

epoch : 62 Current Loss : 0.005601352248775059